In [1]:
import pandas as pd
import numpy as np

import dask
import dask.dataframe as dd
from dask.delayed import delayed

In [2]:
parts = dask.delayed(pd.read_excel)("quina.xlsx")
quina = dd.from_delayed(parts)

In [3]:
quina['Concurso_2'] = quina['Concurso'] 
quina = quina.set_index('Concurso_2')#sort_values(by='Concurso', ascending=True).reset_index(drop=True)

In [4]:
quina = quina.compute()

In [5]:
def jogos_ordenados(bola_1: int, bola_2: int, bola_3: int, bola_4: int, bola_5: int) -> list:
    """Retorna os 15 numeros do resultado da lotofacil, ordenados"""
    return sorted([bola_1, bola_2, bola_3, bola_4, bola_5])

def jogos_desordenados(bola_1: int, bola_2: int, bola_3: int, bola_4: int, bola_5: int) -> list:
    """Retorna os 15 numeros do resultado da lotofacil, desordenados"""
    return  ([bola_1, bola_2, bola_3, bola_4, bola_5])

def  soma_jogos(lista: list) -> int:
    """ retorna soma dos jogos """
    return sum(lista)

def  media_jogos(lista: list) -> int:
    """ retorna media dos jogos """
    return int(np.mean(lista))

def  desvio_padrao_jogos(lista: list) -> int:
    """ retorna desvio padrao dos jogos """
    return int(np.std(lista))

    
# faz a lista dos jogos
quina['jogos_ordenados'] = list(map(jogos_ordenados, quina['bola_1'], quina['bola_2'], quina['bola_3'],
                              quina['bola_4'], quina['bola_5']))

quina['jogos_desordenados'] = list(map(jogos_desordenados, quina['bola_1'], quina['bola_2'], quina['bola_3'],
                              quina['bola_4'], quina['bola_5'])) 

# faz a soma dos jogos e deixa um inteiro
quina['soma_jogos'] = list(map(soma_jogos, quina['jogos_ordenados']))
# faz a média dos jogos e deixa um inteiro
quina['media_jogos'] = list(map(media_jogos, quina['jogos_ordenados']))
# faz o desvio padrão dos jogos
quina['desvio_padrao_jogos'] = list(map(desvio_padrao_jogos, quina['jogos_ordenados']))

quina = quina.drop(['bola_1', 'bola_2', 'bola_3', 'bola_4', 'bola_5'], axis=1)

    

In [6]:
%%time

def entre_numeros(df: pd.DataFrame, primeiro: int, segundo: int, coluna_nova: str):
    """ conta os numeros das bolas entre 1 e 15, 16 e 31 e etc.. """
    contador = 0
    lista_numeros = []
    for jogo in df['jogos_ordenados']:
        for bola in jogo:
            if bola >= primeiro and bola <= segundo:
                contador += 1
        lista_numeros.append(contador)
        contador = 0
    df[coluna_nova] = lista_numeros
    
entre_numeros(quina, 1, 15, "entre_1_e_15")
entre_numeros(quina, 16, 31, "entre_16_e_31")
entre_numeros(quina, 32, 47, "entre_32_e_47")
entre_numeros(quina, 48, 63, "entre_48_e_63")
entre_numeros(quina, 64, 80, "entre_64_e_80")

CPU times: user 21.9 ms, sys: 3.73 ms, total: 25.6 ms
Wall time: 24.7 ms


In [7]:
def contar_bolas(df: pd.DataFrame):
    """ conta os numeros de todas as bolas por jogo e faz a coluna """
    for bola in range(1, 81):
        contador = 0
        lista_numeros = [0 for i in range(len(df) - 150)]
        for jogo in df.iloc[-150:, :]['jogos_ordenados']:
            if bola in jogo:
                contador += 1
            lista_numeros.append(contador)
        coluna_nova = "bola_" + str(bola) + "_sorteio" 
        df[coluna_nova] = lista_numeros
        
contar_bolas(quina)

In [8]:
from matplotlib.pyplot import figure

from matplotlib import pyplot as plt


lista_bola = []
bola = []
for i in range(1,81):
    bola.append(i)
    coluna = 'bola_' + str(i) + "_sorteio"
    lista_bola.append(coluna)
    
xticks = []
for i in range(1,81,2):
    xticks.append(i)

x_bolas = bola
y_values = quina[lista_bola].iloc[-1].values

figure(figsize=(10,5))
axes= plt.axes()

axes.set_xticks(xticks)
#axes.set_yticks([150, 175, 200,225, 250])
plt.tight_layout()
plt.bar(x_bolas, y_values, width=0.5)

<BarContainer object of 80 artists>

In [9]:
quina['jogos_ordenados'].iloc[0]

[25, 45, 60, 76, 79]

In [10]:
%%time
def sorteios_sem_sair_bola(df: pd.DataFrame):
    """ conta o numero de jogos que as bolas ficaram sem ser sorteadas """
    for bola in range(1, 81):
        jogo = 0
        soma = 0
        lista_numeros = []
        for linha in range(len(df['jogos_ordenados'])):
            if bola in df['jogos_ordenados'].iloc[linha]:
                jogo = linha
            soma = df['Concurso'].iloc[linha] - df['Concurso'].iloc[jogo] 
            lista_numeros.append(soma)
        coluna_nova = "bola_" + str(bola) + "_jogos_sem_sorteio" 
        df[coluna_nova] = lista_numeros
        
sorteios_sem_sair_bola(quina)

CPU times: user 19.2 s, sys: 7.01 ms, total: 19.2 s
Wall time: 19.2 s


In [11]:
%%time
def resultado_sorteios_sem_sair_bola(df: pd.DataFrame) -> list:
    """ adiciona a qtd de sorteios que cada bola ficou sem ser sorteada em uma lista """
    lista_jogos = [0]
    for linha in range(1, len(df['jogos_ordenados'])):    
        lista_bolas = []
        for bola in range(1, 81):
            if bola in df['jogos_ordenados'].iloc[linha]:
                lista_bolas.append(df["bola_" + str(bola) + "_jogos_sem_sorteio"].iloc[linha-1]) 
        lista_jogos.append(lista_bolas)
    coluna_nova = 'sorteio_sem_sair_resultados'
    #print(lista_jo)
    df[coluna_nova] = lista_jogos

resultado_sorteios_sem_sair_bola(quina)

CPU times: user 7.17 s, sys: 0 ns, total: 7.17 s
Wall time: 7.17 s


In [25]:
quina['sorteio_sem_sair_resultados']

Concurso_2
1                        0
2          [0, 0, 0, 0, 0]
3          [1, 1, 1, 1, 1]
4          [2, 2, 2, 2, 2]
5          [3, 3, 3, 3, 3]
               ...        
5304     [4, 8, 88, 5, 13]
5305      [1, 2, 2, 38, 3]
5306    [37, 18, 11, 4, 4]
5307     [9, 66, 1, 9, 36]
5308      [1, 32, 7, 9, 1]
Name: sorteio_sem_sair_resultados, Length: 5308, dtype: object

In [12]:
%%time
def soma_jogos_sem_sair(lista: list) -> int:
    """ retorna soma dos jogos """
    if lista == 0:
        return(0)
    else:
        return sum(lista)

def media_jogos_sem_sair(lista: list) -> int:
    """ retorna media dos jogos """
    if lista == 0:
        return(0)
    else:
        return round(np.mean(lista), 2)

def desvio_padrao_jogos_sem_sair(lista: list) -> int:
    """ retorna desvio padrao dos jogos """
    if lista == 0:
        return(0)
    else:
        return round(np.std(lista), 2)

quina['sorteio_sem_sair_resultados_sum'] = list(map(soma_jogos_sem_sair, 
                                                        quina['sorteio_sem_sair_resultados']))
quina['sorteio_sem_sair_resultados_mean'] = list(map(media_jogos_sem_sair, 
                                                        quina['sorteio_sem_sair_resultados']))
quina['sorteio_sem_sair_resultados_std'] = list(map(desvio_padrao_jogos_sem_sair, 
                                                        quina['sorteio_sem_sair_resultados']))

CPU times: user 466 ms, sys: 24 ms, total: 490 ms
Wall time: 472 ms


In [18]:
quina['sorteio_sem_sair_resultados_sum'].iloc[-10:].value_counts()

63     1
126    1
93     1
61     1
74     1
121    1
88     1
118    1
46     1
50     1
Name: sorteio_sem_sair_resultados_sum, dtype: int64

In [13]:
%%time
def numeros_primos(df: pd.DataFrame):
    """ faz uma coluna com a quantidade de numeros primos por jogo"""
    primos = [1,2,3,5,7,11,13,17,19,23,29,31,37,41,43,47,53,59,61,67,71,73,79]
    qtd_primos = []
    for jogo in df['jogos_ordenados']:
        contador = 0
        for bola in jogo:
            if bola in primos:
                contador += 1
        qtd_primos.append(contador)
    df['qtd_primos'] = qtd_primos

numeros_primos(quina)

CPU times: user 18.1 ms, sys: 0 ns, total: 18.1 ms
Wall time: 17.5 ms


In [14]:
%%time
def numeros_fibonacci(df: pd.DataFrame):
    """ faz uma coluna com a quantidade de numeros de fibonacci por jogo"""
    fibonacci = [1, 2, 3, 5, 8, 13, 21, 34, 55]
    qtd_fibonacci = []
    for jogo in df['jogos_ordenados']:
        contador = 0
        for bola in jogo:
            if bola in fibonacci:
                contador += 1
        qtd_fibonacci.append(contador)
    df['qtd_fibonacci'] = qtd_fibonacci

numeros_fibonacci(quina)

CPU times: user 11 ms, sys: 0 ns, total: 11 ms
Wall time: 11.2 ms


In [15]:
%%time
def numeros_fibonacci_e_primo(df: pd.DataFrame):
    """ faz uma coluna com a quantidade de numeros de fibonacci e primos por jogo"""
    fibonacci_primo = [1, 2, 3, 5,13]
    qtd_fibonacci_primo = []
    for jogo in df['jogos_ordenados']:
        contador = 0
        for bola in jogo:
            if bola in fibonacci_primo:
                contador += 1
        qtd_fibonacci_primo.append(contador)
    df['qtd_fibonacci_primo'] = qtd_fibonacci_primo

numeros_fibonacci_e_primo(quina)

CPU times: user 8 ms, sys: 0 ns, total: 8 ms
Wall time: 7.89 ms


In [16]:
%%time
def pares_impares(df: pd.DataFrame):
    """ cria 2 colunas com as quantidades de numeros pares e impares por jogo"""
    pares = []
    impares = []
    for jogo in df['jogos_ordenados']:
        contador_par = 0
        contador_impar = 0
        for bola in jogo:
            if bola % 2 == 0:
                contador_par += 1
            else:
                contador_impar += 1
        pares.append(contador_par)
        impares.append(contador_impar)
    df['qtd_numeros_pares'] = pares
    df['qtd_numeros_impares'] = impares

    
pares_impares(quina)

CPU times: user 16.8 ms, sys: 0 ns, total: 16.8 ms
Wall time: 16.3 ms


In [17]:
%%time
def divisivel_por_tres(df: pd.DataFrame):
    """ cria coluna com a quantidade de numeros divisiveis por 3"""
    divisivel_tres = []
    for jogo in df['jogos_ordenados']:
        contador = 0
        for bola in jogo:
            if bola % 3 == 0:
                contador += 1
        divisivel_tres.append(contador)
    df['qtd_divisivel_tres'] = divisivel_tres

divisivel_por_tres(quina)

CPU times: user 7.23 ms, sys: 0 ns, total: 7.23 ms
Wall time: 7.09 ms


In [51]:
%%time
def diferenca_entre_bolas(df: pd.DataFrame) -> list:
    """ adiciona a diferenca entre bolas em uma lista """
    lista_jogos = []
    for linha in range(len(df['jogos_ordenados'])):    
        lista_diferenca = []
        for bola in range(1, 5):
            lista_diferenca.append(df['jogos_ordenados'].iloc[linha][bola] - df['jogos_ordenados'].iloc[linha][bola-1])

        lista_jogos.append(lista_diferenca)
    coluna_nova = 'diferenca_entre_bolas'
    #print(lista_jo)
    df[coluna_nova] = lista_jogos

diferenca_entre_bolas(quina)

CPU times: user 574 ms, sys: 11.3 ms, total: 586 ms
Wall time: 786 ms


In [52]:
quina[['jogos_ordenados', 'diferenca_entre_bolas']]

,jogos_ordenados,diferenca_entre_bolas
Concurso_2,,
1,"[25, 45, 60, 76, 79]","[20, 15, 16, 3]"
2,"[13, 30, 58, 63, 64]","[17, 28, 5, 1]"
3,"[5, 15, 32, 33, 48]","[10, 17, 1, 15]"
4,"[27, 57, 60, 61, 77]","[30, 3, 1, 16]"
5,"[19, 44, 53, 54, 71]","[25, 9, 1, 17]"
...,...,...
5304,"[7, 8, 42, 46, 56]","[1, 34, 4, 10]"
5305,"[23, 30, 53, 58, 80]","[7, 23, 5, 22]"
5306,"[11, 14, 49, 68, 69]","[3, 35, 19, 1]"


In [53]:
quina['diferenca_entre_bolas_sum'] = list(map(soma_jogos_sem_sair, 
                                                        quina['diferenca_entre_bolas']))
quina['diferenca_entre_bolas_mean'] = list(map(media_jogos_sem_sair, 
                                                        quina['diferenca_entre_bolas']))
quina['diferenca_entre_bolas_std'] = list(map(desvio_padrao_jogos_sem_sair, 
                                                        quina['diferenca_entre_bolas']))

In [58]:
%%time
def coluna_diferenca_bolas(df: pd.DataFrame) -> list:
    """ faz colunas com as diferencas entre bolas """
    for bola in range(4):    
        lista_diferenca = []
        for linha in range(len(df['jogos_ordenados'])):
            lista_diferenca.append(df['diferenca_entre_bolas'].iloc[linha][bola])

        coluna_nova = 'diferenca_entre_bolas_' + str(bola)
        #print(len(lista_diferenca))
        df[coluna_nova] = lista_diferenca

coluna_diferenca_bolas(quina)

CPU times: user 275 ms, sys: 0 ns, total: 275 ms
Wall time: 274 ms


In [60]:
quina[['diferenca_entre_bolas','diferenca_entre_bolas_0', 'diferenca_entre_bolas_1',
       'diferenca_entre_bolas_2', 'diferenca_entre_bolas_3']]

,diferenca_entre_bolas,diferenca_entre_bolas_0,diferenca_entre_bolas_1,diferenca_entre_bolas_2,diferenca_entre_bolas_3
Concurso_2,,,,,
1,"[20, 15, 16, 3]",20,15,16,3
2,"[17, 28, 5, 1]",17,28,5,1
3,"[10, 17, 1, 15]",10,17,1,15
4,"[30, 3, 1, 16]",30,3,1,16
5,"[25, 9, 1, 17]",25,9,1,17
...,...,...,...,...,...
5304,"[1, 34, 4, 10]",1,34,4,10
5305,"[7, 23, 5, 22]",7,23,5,22
5306,"[3, 35, 19, 1]",3,35,19,1


In [ ]:
soma, media, desvio padrao, entre numeros, primos, fibonacci, primos e fibo, pares, impares, divisivel_3

In [18]:
%%time
def resultado_jogo(soma: int, media: int, desvio_padrao: int, entre_1_5: int, entre_6_10: int, 
                   entre_11_15: int, entre_16_20: int, entre_21_25: int, 
                   primos: int, fibonacci: int, primos_fibo: int, pares: int, impares: int, divisivel_3: int,
                   sem_sair_sum: int):
    
    return ([soma, media, desvio_padrao, entre_1_5, entre_6_10, entre_11_15, entre_16_20, entre_21_25,
             primos, fibonacci, primos_fibo, pares, impares, divisivel_3, sem_sair_sum])

quina['resultados'] = list(map(resultado_jogo, quina['soma_jogos'], quina['media_jogos'], 
                                   quina['desvio_padrao_jogos'],
                              quina['entre_1_e_15'], quina['entre_16_e_31'], quina['entre_32_e_47'],
                              quina['entre_48_e_63'], quina['entre_64_e_80'], quina['qtd_primos'],
                              quina['qtd_fibonacci'], quina['qtd_fibonacci_primo'],
                                   quina['qtd_numeros_pares'],
                              quina['qtd_numeros_impares'], quina['qtd_divisivel_tres'], 
                                   quina['sorteio_sem_sair_resultados_sum']))


CPU times: user 14.4 ms, sys: 3.97 ms, total: 18.3 ms
Wall time: 17.7 ms


In [25]:
#soma, media, desvio padrao, entre numeros, primos, fibonacci, primos e fibo, pares, impares, divisivel_3
quina['resultados'].iloc[0]

[285, 57, 20, 0, 1, 1, 1, 2, 1, 0, 0, 2, 3, 2, 0]

In [19]:
%%time
# Criar todos os jogos possiveis
numeros_possiveis = []
for i in range (1,81):
    numeros_possiveis.append(i)

import itertools
jogos_possiveis = list(itertools.combinations(numeros_possiveis, 5))

jogos_possiveis = [list(i) for i in jogos_possiveis]

data_jogos_parts = dask.delayed(pd.DataFrame)()
data_jogos = dd.from_delayed(data_jogos_parts)
data_jogos = data_jogos.compute()
data_jogos['jogos_ordenados'] = jogos_possiveis

CPU times: user 49.8 s, sys: 2.22 s, total: 52 s
Wall time: 51.9 s


In [62]:
data_jogos['jogos_ordenados'][0]

[1, 2, 3, 4, 5]

In [20]:
%%time
# faz a soma dos jogos e deixa um inteiro
data_jogos['soma_jogos'] = list(map(soma_jogos, data_jogos['jogos_ordenados']))
# faz a média dos jogos e deixa um inteiro
data_jogos['media_jogos'] = list(map(media_jogos, data_jogos['jogos_ordenados']))
# faz o desvio padrão dos jogos
data_jogos['desvio_padrao_jogos'] = list(map(desvio_padrao_jogos, data_jogos['jogos_ordenados']))

entre_numeros(data_jogos, 1, 5, "entre_1_e_15")
entre_numeros(data_jogos, 6, 10, "entre_16_e_31")
entre_numeros(data_jogos, 11, 15, "entre_32_e_47")
entre_numeros(data_jogos, 16, 20, "entre_48_e_63")
entre_numeros(data_jogos, 21, 25, "entre_64_e_80")

numeros_primos(data_jogos)
numeros_fibonacci(data_jogos)

numeros_fibonacci_e_primo(data_jogos)
pares_impares(data_jogos)

divisivel_por_tres(data_jogos)


CPU times: user 27min 44s, sys: 1min 22s, total: 29min 6s
Wall time: 27min 29s


In [62]:
%%time
diferenca_entre_bolas(data_jogos)

data_jogos['diferenca_entre_bolas_sum'] = list(map(soma_jogos_sem_sair, 
                                                        data_jogos['diferenca_entre_bolas']))
data_jogos['diferenca_entre_bolas_mean'] = list(map(media_jogos_sem_sair, 
                                                        data_jogos['diferenca_entre_bolas']))
data_jogos['diferenca_entre_bolas_std'] = list(map(desvio_padrao_jogos_sem_sair, 
                                                        data_jogos['diferenca_entre_bolas']))

CPU times: user 1h 35min 21s, sys: 1min 57s, total: 1h 37min 19s
Wall time: 1h 36min 46s


In [64]:
%%time
def coluna_diferenca_bolas(df: pd.DataFrame) -> list:
    """ faz colunas com as diferencas entre bolas """
    for bola in range(4):    
        lista_diferenca = []
        for linha in range(len(df['jogos_ordenados'])):
            lista_diferenca.append(df['diferenca_entre_bolas'].iloc[linha][bola])

        coluna_nova = 'diferenca_entre_bolas_' + str(bola)
        #print(len(lista_diferenca))
        df[coluna_nova] = lista_diferenca

coluna_diferenca_bolas(data_jogos)

CPU times: user 26min 20s, sys: 7.47 s, total: 26min 27s
Wall time: 26min 41s


In [ ]:
---------------------------------------------

In [26]:
#data_jogos.to_csv('quina_data_jogos.csv')
#parts = dask.delayed(pd.read_excel)("quina.xlsx")
#quina = dd.from_delayed(parts)
data_jogos_parts = dask.delayed(pd.read_csv)('quina_data_jogos.csv')
data_jogos = dd.from_delayed(data_jogos_parts)

In [29]:
%%time
data_jogos = data_jogos.compute()

In [ ]:
%%time
# Criar todos os jogos possiveis
numeros_possiveis = []
for i in range (1,81):
    numeros_possiveis.append(i)

import itertools
jogos_possiveis = list(itertools.combinations(numeros_possiveis, 5))

jogos_possiveis = [list(i) for i in jogos_possiveis]

#data_jogos_parts = dask.delayed(pd.DataFrame)()
#data_jogos = dd.from_delayed(data_jogos_parts)
#data_jogos = data_jogos.compute()
data_jogos['jogos_ordenados'] = jogos_possiveis

In [ ]:
-----------------------------------------------------------------------

In [21]:
%%time
import dask.array as da

def resultado_sorteios_sem_sair_bola_2(df: pd.DataFrame, df_2: pd.DataFrame) -> list:
    """ adiciona a qtd de sorteios que cada bola ficou sem ser sorteada em uma lista """
    
    sorteios_sem_bola = list(df[["bola_1_jogos_sem_sorteio", "bola_2_jogos_sem_sorteio", "bola_3_jogos_sem_sorteio",
           "bola_4_jogos_sem_sorteio", "bola_5_jogos_sem_sorteio",
                                "bola_6_jogos_sem_sorteio", "bola_7_jogos_sem_sorteio", "bola_8_jogos_sem_sorteio",
           "bola_9_jogos_sem_sorteio", "bola_10_jogos_sem_sorteio",
                                "bola_11_jogos_sem_sorteio", "bola_12_jogos_sem_sorteio", "bola_13_jogos_sem_sorteio",
           "bola_14_jogos_sem_sorteio", "bola_15_jogos_sem_sorteio",
                                "bola_16_jogos_sem_sorteio", "bola_17_jogos_sem_sorteio", "bola_18_jogos_sem_sorteio",
           "bola_19_jogos_sem_sorteio", "bola_20_jogos_sem_sorteio",
                                "bola_21_jogos_sem_sorteio", "bola_22_jogos_sem_sorteio", "bola_23_jogos_sem_sorteio",
           "bola_24_jogos_sem_sorteio", "bola_25_jogos_sem_sorteio",
                                "bola_26_jogos_sem_sorteio", "bola_27_jogos_sem_sorteio", "bola_28_jogos_sem_sorteio",
           "bola_29_jogos_sem_sorteio", "bola_30_jogos_sem_sorteio",
                                "bola_31_jogos_sem_sorteio", "bola_32_jogos_sem_sorteio", "bola_33_jogos_sem_sorteio",
           "bola_34_jogos_sem_sorteio", "bola_35_jogos_sem_sorteio",
                                "bola_36_jogos_sem_sorteio", "bola_37_jogos_sem_sorteio", "bola_38_jogos_sem_sorteio",
           "bola_39_jogos_sem_sorteio", "bola_40_jogos_sem_sorteio",
                                "bola_41_jogos_sem_sorteio", "bola_42_jogos_sem_sorteio", "bola_43_jogos_sem_sorteio",
           "bola_44_jogos_sem_sorteio", "bola_45_jogos_sem_sorteio",
                                "bola_46_jogos_sem_sorteio", "bola_47_jogos_sem_sorteio", "bola_48_jogos_sem_sorteio",
           "bola_49_jogos_sem_sorteio", "bola_50_jogos_sem_sorteio",
                                "bola_51_jogos_sem_sorteio", "bola_52_jogos_sem_sorteio", "bola_53_jogos_sem_sorteio",
           "bola_54_jogos_sem_sorteio", "bola_55_jogos_sem_sorteio",
                                "bola_56_jogos_sem_sorteio", "bola_57_jogos_sem_sorteio", "bola_58_jogos_sem_sorteio",
           "bola_59_jogos_sem_sorteio", "bola_60_jogos_sem_sorteio",
                                "bola_61_jogos_sem_sorteio", "bola_62_jogos_sem_sorteio", "bola_63_jogos_sem_sorteio",
           "bola_64_jogos_sem_sorteio", "bola_65_jogos_sem_sorteio",
                                "bola_66_jogos_sem_sorteio", "bola_67_jogos_sem_sorteio", "bola_68_jogos_sem_sorteio",
           "bola_69_jogos_sem_sorteio", "bola_70_jogos_sem_sorteio",
                                "bola_71_jogos_sem_sorteio", "bola_72_jogos_sem_sorteio", "bola_73_jogos_sem_sorteio",
           "bola_74_jogos_sem_sorteio", "bola_75_jogos_sem_sorteio",
                                "bola_76_jogos_sem_sorteio", "bola_77_jogos_sem_sorteio", "bola_78_jogos_sem_sorteio",
           "bola_79_jogos_sem_sorteio", "bola_80_jogos_sem_sorteio"]].iloc[-1])
    
    sorteios_sem_bola = dask.delayed(sorteios_sem_bola)
    sorteios_sem_bola = sorteios_sem_bola.compute()
    
    lista_jogos = []
    lista_jogos = dask.delayed(lista_jogos)
    lista_jogos = lista_jogos.compute()
    
    for linha in range(len(df_2['jogos_ordenados'])):    
        lista_bolas = []
        lista_bolas = dask.delayed(lista_bolas)
        lista_bolas = lista_bolas.compute()
        
        for bola in df_2['jogos_ordenados'].iloc[linha]:
            lista_bolas.append(sorteios_sem_bola[bola-1])
        lista_jogos.append(lista_bolas)
    coluna_nova = 'sorteio_sem_sair_resultados'
    df_2[coluna_nova] = lista_jogos

resultado_sorteios_sem_sair_bola_2(quina, data_jogos)

CPU times: user 1h 45min 21s, sys: 1min 45s, total: 1h 47min 7s
Wall time: 1h 46min 51s


In [68]:
data_jogos

,Unnamed: 0,jogos_ordenados,soma_jogos,media_jogos,desvio_padrao_jogos,entre_1_e_15,entre_16_e_31,entre_32_e_47,entre_48_e_63,entre_64_e_80,qtd_primos,qtd_fibonacci,qtd_fibonacci_primo,qtd_numeros_pares,qtd_numeros_impares,qtd_divisivel_tres,sorteio_sem_sair_resultados
0,0,"[1, 2, 3, 4, 5]",15,3,1,5,0,0,0,0,4,4,4,2,3,1,"[19, 15, 31, 1, 1]"
1,1,"[1, 2, 3, 4, 6]",16,3,1,4,1,0,0,0,3,3,3,3,2,2,"[19, 15, 31, 1, 14]"
2,2,"[1, 2, 3, 4, 7]",17,3,2,4,1,0,0,0,4,3,3,2,3,1,"[19, 15, 31, 1, 4]"
3,3,"[1, 2, 3, 4, 8]",18,3,2,4,1,0,0,0,3,4,3,3,2,1,"[19, 15, 31, 1, 4]"
4,4,"[1, 2, 3, 4, 9]",19,3,2,4,1,0,0,0,3,3,3,2,3,2,"[19, 15, 31, 1, 19]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24040011,24040011,"[75, 76, 77, 78, 80]",386,77,1,0,0,0,0,0,0,0,0,3,2,2,"[7, 10, 26, 8, 3]"
24040012,24040012,"[75, 76, 77, 79, 80]",387,77,1,0,0,0,0,0,1,0,0,2,3,1,"[7, 10, 26, 5, 3]"
24040013,24040013,"[75, 76, 78, 79, 80]",388,77,1,0,0,0,0,0,1,0,0,3,2,2,"[7, 10, 8, 5, 3]"
24040014,24040014,"[75, 77, 78, 79, 80]",389,77,1,0,0,0,0,0,1,0,0,2,3,2,"[7, 26, 8, 5, 3]"


In [121]:
data_jogos[['sorteio_sem_sair_resultados', 'jogos_ordenados']]

,sorteio_sem_sair_resultados,jogos_ordenados
0,"[1, 0, 0, 0, 0, 0, 1, 1, 2, 0, 0, 0, 0, 2, 0]","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
1,"[1, 0, 0, 0, 0, 0, 1, 1, 2, 0, 0, 0, 0, 2, 0]","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
2,"[1, 0, 0, 0, 0, 0, 1, 1, 2, 0, 0, 0, 0, 2, 0]","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
3,"[1, 0, 0, 0, 0, 0, 1, 1, 2, 0, 0, 0, 0, 2, 1]","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
4,"[1, 0, 0, 0, 0, 0, 1, 1, 2, 0, 0, 0, 0, 2, 0]","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
...,...,...
3268755,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 3, 0, 0, 1, 1, 3]","[10, 11, 12, 13, 15, 16, 17, 18, 19, 20, 21, 2..."
3268756,"[0, 0, 0, 2, 0, 0, 0, 1, 0, 3, 0, 0, 1, 1, 3]","[10, 11, 12, 14, 15, 16, 17, 18, 19, 20, 21, 2..."
3268757,"[0, 0, 0, 2, 0, 0, 0, 1, 0, 3, 0, 0, 1, 1, 3]","[10, 11, 13, 14, 15, 16, 17, 18, 19, 20, 21, 2..."
3268758,"[0, 0, 0, 2, 0, 0, 0, 1, 0, 3, 0, 0, 1, 1, 3]","[10, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 2..."


In [22]:
%%time
data_jogos['sorteio_sem_sair_resultados_sum'] = list(map(soma_jogos_sem_sair, 
                                                        data_jogos['sorteio_sem_sair_resultados']))
data_jogos['sorteio_sem_sair_resultados_mean'] = list(map(media_jogos_sem_sair, 
                                                        data_jogos['sorteio_sem_sair_resultados']))
data_jogos['sorteio_sem_sair_resultados_std'] = list(map(desvio_padrao_jogos_sem_sair, 
                                                        data_jogos['sorteio_sem_sair_resultados']))

CPU times: user 30min 21s, sys: 1min 20s, total: 31min 41s
Wall time: 30min 1s


In [46]:
quina['sorteio_sem_sair_resultados_sum'].iloc[-300:].value_counts().head(50)

63     9
62     8
50     8
64     7
75     7
42     7
59     6
45     6
102    5
70     5
88     5
78     5
69     5
46     5
61     5
49     4
83     4
77     4
80     4
55     4
51     4
97     4
58     4
33     4
93     4
29     4
27     4
107    4
54     3
56     3
57     3
86     3
87     3
60     3
104    3
109    3
39     3
68     3
30     3
72     3
73     3
36     3
74     3
38     3
40     3
129    3
126    3
76     3
81     3
47     3
Name: sorteio_sem_sair_resultados_sum, dtype: int64

In [1]:
data_jogos['sorteio_sem_sair_resultados_sum'].value_counts()

NameError: name 'data_jogos' is not defined

In [38]:
len(data_jogos[data_jogos['sorteio_sem_sair_resultados_sum'] == 80])

284690

In [65]:
data_jogos[['sorteio_sem_sair_resultados_sum', 'soma_jogos','media_jogos','desvio_padrao_jogos','entre_1_e_15',
            'entre_16_e_31',
           'entre_32_e_47', 'entre_48_e_63', 'entre_64_e_80', 
           'qtd_primos', 'qtd_fibonacci', 'qtd_fibonacci_primo', 'qtd_numeros_pares', 
           'qtd_numeros_impares', 'qtd_divisivel_tres', 'diferenca_entre_bolas_sum',
           'diferenca_entre_bolas_std', 'diferenca_entre_bolas_mean', 
           'diferenca_entre_bolas_0', 'diferenca_entre_bolas_1',
           'diferenca_entre_bolas_2', 'diferenca_entre_bolas_3']].corr().style.background_gradient(cmap='coolwarm')

,sorteio_sem_sair_resultados_sum,soma_jogos,media_jogos,desvio_padrao_jogos,entre_1_e_15,entre_16_e_31,entre_32_e_47,entre_48_e_63,entre_64_e_80,qtd_primos,qtd_fibonacci,qtd_fibonacci_primo,qtd_numeros_pares,qtd_numeros_impares,qtd_divisivel_tres,diferenca_entre_bolas_sum,diferenca_entre_bolas_std,diferenca_entre_bolas_mean,diferenca_entre_bolas_0,diferenca_entre_bolas_1,diferenca_entre_bolas_2,diferenca_entre_bolas_3
sorteio_sem_sair_resultados_sum,1.000000,-0.063897,-0.063871,-0.061850,-0.003254,-0.037962,0.079178,0.040131,-0.020608,0.007773,0.010137,0.144256,-0.098718,0.098718,0.086773,-0.055767,-0.050020,-0.055767,-0.025968,-0.035964,-0.020780,0.012172
soma_jogos,-0.063897,1.000000,0.999605,-0.000000,-0.419296,-0.363389,-0.307483,-0.251577,-0.195671,-0.166843,-0.381166,-0.399169,0.021652,-0.021652,0.000000,0.000000,0.000000,0.000000,0.392792,0.130931,-0.130931,-0.392792
media_jogos,-0.063871,0.999605,1.000000,0.000004,-0.419130,-0.363246,-0.307362,-0.251478,-0.195594,-0.166777,-0.381016,-0.399012,0.021644,-0.021644,-0.000000,-0.000000,-0.000000,-0.000000,0.392637,0.130879,-0.130879,-0.392637
desvio_padrao_jogos,-0.061850,-0.000000,0.000004,1.000000,0.350236,0.218599,0.100115,-0.003032,-0.088648,0.124465,0.206863,0.305635,0.000000,-0.000000,-0.028605,0.950747,0.650333,0.950747,0.151149,0.450156,0.450156,0.151149
entre_1_e_15,-0.003254,-0.419296,-0.419130,0.350236,1.000000,-0.066667,-0.066667,-0.066667,-0.066667,0.292373,0.561780,0.786667,-0.051640,0.051640,-0.068908,0.349741,0.239194,0.349741,0.042657,0.129977,0.134829,0.134928
entre_16_e_31,-0.037962,-0.363389,-0.363246,0.218599,-0.066667,1.000000,-0.066667,-0.066667,-0.066667,-0.049917,0.071499,-0.066667,0.051640,-0.051640,0.041345,0.185617,0.194410,0.185617,-0.120186,0.088917,0.131257,0.134801
entre_32_e_47,0.079178,-0.307483,-0.307362,0.100115,-0.066667,-0.066667,1.000000,-0.066667,-0.066667,0.064179,0.071499,0.146667,-0.051640,0.051640,0.041345,0.062245,0.116131,0.062245,-0.193023,0.021274,0.116865,0.133619
entre_48_e_63,0.040131,-0.251577,-0.251478,-0.003032,-0.066667,-0.066667,-0.066667,1.000000,-0.066667,0.064179,-0.091928,-0.066667,0.051640,-0.051640,-0.068908,-0.028363,0.025018,-0.028363,-0.204189,-0.049287,0.088194,0.129405
entre_64_e_80,-0.020608,-0.195671,-0.195594,-0.088648,-0.066667,-0.066667,-0.066667,-0.066667,1.000000,-0.049917,0.071499,-0.066667,-0.051640,0.051640,0.041345,-0.092862,-0.061019,-0.092862,-0.176402,-0.106961,0.046277,0.119625
qtd_primos,0.007773,-0.166843,-0.166777,0.124465,0.292373,-0.049917,0.064179,0.064179,-0.049917,1.000000,0.210866,0.406469,-0.579987,0.579987,-0.381808,0.121092,0.089136,0.121092,-0.007231,0.040759,0.058618,0.061025


In [46]:
%%time
# Criar todos os jogos possiveis
numeros_possiveis = []
for i in range (1,7):
    numeros_possiveis.append(i)

import itertools
jogos_possiveis = list(itertools.combinations(numeros_possiveis, 5))

jogos_possiveis = [list(i) for i in jogos_possiveis]

#data_jogos_parts = dask.delayed(pd.DataFrame)()
#data_jogos = dd.from_delayed(data_jogos_parts)
#data_jogos = data_jogos.compute()
#data_jogos['jogos_ordenados'] = jogos_possiveis

CPU times: user 15 µs, sys: 0 ns, total: 15 µs
Wall time: 20 µs


In [47]:
jogos_possiveis

[[1, 2, 3, 4, 5],
 [1, 2, 3, 4, 6],
 [1, 2, 3, 5, 6],
 [1, 2, 4, 5, 6],
 [1, 3, 4, 5, 6],
 [2, 3, 4, 5, 6]]

In [66]:
data_jogos.to_csv('data_jogos_quina.csv')

In [67]:
len(data_jogos)

24040016

In [2]:
data_jogos

NameError: name 'data_jogos' is not defined